# Метод Франка-Вульфа к поиску равновесия в модели Бэкмана

## Импорт библиотек

In [12]:
import numpy as np
import codecs
import re

## Чтение данных

In [13]:
data_path = '../data/SiouxFalls_trips.txt'

massiv = np.zeros((24,24))
with codecs.open(data_path, 'r', 'utf-8') as fin:
    for line_idx, line in enumerate(fin):
        row = line.strip('\n').split(' ')
        if len(row) == 3:
            i = int(row[1])-1
            j = 0
        else:
            for x in row:
                t = re.findall(r'(\w+.0);', x)
                if len(t) != 0:
                    massiv[i][j] = float(t[0]) 
                    j += 1
massiv = massiv.tolist()


data_path2 = '../data/SiouxFalls_net.txt'

init = []
term = []
A = []
C = []

with codecs.open(data_path2, 'r', 'utf-8') as fin:
    for line_idx, line in enumerate(fin):
        if line_idx == 0:
            continue
        row = line.strip('\n').split('\t')

        init.append(int(row[1])-1)
        term.append(int(row[2])-1)
        A.append(int(row[5]))
        if int(row[7]) == 0:
            C.append(0)
        else:
            C.append(int(row[5]) * float(row[6]) / (float(row[3]) ** int(row[7])))

##  Начальные данные

In [14]:
nodes = 24
Links = 76

t = [0]*Links
f = [0]*Links

epsilon = 0
cr_min = 10**10

## Метод Франка-Вульфа

В качестве критерия останова используем: $\min\limits_{k}(\nabla \Psi(f^k),f^k-y^k) \leq \epsilon$

In [15]:
#Номер итерации 
k = 0
#Проверяем критерий останова 
while cr_min > epsilon:

    #Массив для хранения текущего решения
    y = [0]*Links

    #Удельные затраты на проезд по дуге e
    for e in range(Links):
        t[e] = A[e] + C[e]*pow(f[e],4);

    #Создаем массив весов ребер для алгоритма Дейкстры
    w = [0]*nodes
    for i in range(nodes):
        w[i] = [10 ** 10]*nodes

    for e in range(Links):
        w[init[e]][term[e]] = t[e];

    #Алгоритм Дейкстры (сложность O(n^2))
    for i in range(nodes):
        
        INF = 10 ** 10
        p = [-1] * nodes
        dist = [INF] * nodes
        dist[i] = 0
        used = [False] * nodes
        min_dist = 0
        min_vertex = i
        while min_dist < INF:
            s = min_vertex 
            used[s] = True 
            for j in range(nodes): 
                if dist[s] + w[s][j] < dist[j]: 
                    dist[j] = dist[s] + w[s][j] 
                    p[j] = s;
            min_dist = INF
            for j in range(nodes):
                if not used[j] and dist[j] < min_dist:
                    min_dist = dist[j]
                    min_vertex = j

        # Cтроим решение y[k]
        for j in range(nodes):
            s = p[j]
            ss = j
            while (s!=-1):
                for e in range(Links):
                    if ((init[e] == s) and (term[e] == ss)):
                        y[e] = y[e] + massiv[i][j]
                ss = p[ss]
                s = p[s]



    #Вычисляем параметры для критерия останова
    if k >= 1:
        cr = 0
        for e in range(Links):
            cr = cr + t[e] * (f[e] - y[e])

        if cr < cr_min:
            cr_min = cr; 


    if k > 20: 
        cr_min = cr_min / 1000
    if (k == 0):
        for e in range(Links):
            f[e] = y[e]
    else:
        gamma = 2 / (k + 1)
        for e in range(Links):
            f[e] = (1 - gamma) * f[e] + gamma * y[e]


    if k == 0:
        epsilon = 0
        for e in range(Links):
            epsilon = epsilon + A[e] * f[e] + C[e] * (f[e] ** 5) / 5.
        epsilon = 0.01 * epsilon

    #Новый шаг
    k = k + 1

## Вывод результатов

In [16]:
print ('Дуга: оптимальная величина потока\n')
for e in range(Links):
	print ('%d %d: f = %d' % (init[e], term[e], f[e]))

Дуга: оптимальная величина потока

0 1: f = 4800
0 2: f = 12900
1 0: f = 4900
1 5: f = 1600
2 0: f = 12800
2 3: f = 32300
2 11: f = 22400
3 2: f = 30800
3 4: f = 36200
3 10: f = 11900
4 3: f = 34800
4 5: f = 6000
4 8: f = 42500
5 1: f = 1700
5 4: f = 5900
5 7: f = 0
6 7: f = 22100
6 17: f = 19000
7 5: f = 0
7 6: f = 22700
7 8: f = 23800
7 15: f = 0
8 4: f = 41200
8 7: f = 24400
8 9: f = 35500
9 8: f = 34900
9 10: f = 10500
9 14: f = 53400
9 15: f = 0
9 16: f = 23400
10 3: f = 11900
10 9: f = 10400
10 11: f = 0
10 13: f = 0
11 2: f = 23800
11 10: f = 0
11 12: f = 15600
12 11: f = 17100
12 23: f = 10500
13 10: f = 0
13 14: f = 18100
13 22: f = 2100
14 9: f = 52800
14 13: f = 6400
14 18: f = 33300
14 21: f = 17600
15 7: f = 0
15 9: f = 0
15 16: f = 0
15 17: f = 26100
16 9: f = 23400
16 15: f = 0
16 18: f = 0
17 6: f = 18400
17 15: f = 26100
17 19: f = 24700
18 14: f = 38500
18 16: f = 0
18 19: f = 32200
19 17: f = 24000
19 18: f = 37400
19 20: f = 9200
19 21: f = 17600
20 19: f = 11000
20